# IA – Projeto 02
## Clustering – Perfis de Desempenho em corridas de F1

**Unidade Curricular:** Inteligência Artificial  
**Ano Letivo:** 2025/2026  

### Discentes

- Carlos Sousa (24880)  
- Pedro Gonçalves (26018)  
- Carlos Moreda (26875)  

---

## 1. Introdução e Objetivo

Neste Notebook 2 – Clustering vamos usar também os dados de resultados de Fórmula 1 utilizando o dataset **"Formula 1 Race Data (1950–2017)"** do Kaggle, mas agora com uma perspetiva **não supervisionada**.

Dataset original: **[Formula 1 Race Data (Kaggle)](https://www.kaggle.com/datasets/cjgdev/formula-1-race-data-19502017)**

Em vez de tentar prever diretamente se um piloto termina no pódio, o objetivo aqui é:

>Identificar grupos de perfis de desempenho em corrida de pilotos, com base em variáveis numéricas relacionadas com grelha de partida, posição final, pontos, voltas e tempos.

A ideia é descobrir automaticamente padrões como, por exemplo:

- Corridas **“dominantes”** – partir à frente e terminar à frente, com muitos pontos;
- Corridas **“de recuperação”** – partir de trás e ganhar muitas posições;
- Corridas **“anónimas”** – meio da tabela, poucos pontos;

Ou seja, cada linha continua a ser um resultado de piloto numa corrida, mas agora queremos agrupar esses resultados em tipos de corrida com comportamento semelhante.

Posto isto neste notebook vamos:

1. Definir o objetivo de negócio;  
2. Preparar os dados para clustering (seleção de variáveis, tratamento de faltas, normalização);  
3. Aplicar o algoritmo **K-Means** para diferentes valores de k;  
4. Otimizar o número de clusters (análise de **elbow** e **silhouette**);  
5. Analisar as características de cada cluster, incluindo:
   - número de elementos por cluster;
   - média, desvio padrão, mínimo e máximo das variáveis numéricas;
   - distribuição de alguns atributos categóricos (equipa, país do circuito, pódio / não pódio).

---


#### Imports e Configuração Inicial

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

sns.set(style="whitegrid")
%matplotlib inline

#### Carregamento dos datasets de F1

In [ ]:
# Carregar ficheiro de resultados
results = pd.read_csv("results.csv")

# Carregar ficheiros de contexto
races = pd.read_csv("races.csv")                                   # informação da corrida (ano, circuito, data,etc)
drivers = pd.read_csv("drivers.csv", encoding="latin1")            # info dos pilotos
constructors = pd.read_csv("constructors.csv", encoding="latin1")  # info das equipas
circuits = pd.read_csv("circuits.csv", encoding="latin1")          # info dos circuitos

#### Junção de Informação de Corrida e Circuito

In [ ]:
# Adicionar ano, ronda e circuito
df = results.merge(
    races[["raceId", "year", "round", "circuitId"]],
    on="raceId",
    how="left"
)

# Adicionar país do circuito
df = df.merge(
    circuits[["circuitId", "country"]],
    on="circuitId",
    how="left"
)

# Ver as primeiras linhas do dataset final
df.head()

#### Limpeza mínima e variáveis auxiliares

In [ ]:
# Remover linhas sem posição final
df = df.dropna(subset=["positionOrder"])

# Garantir que a coluna positionOrder é inteira
df["positionOrder"] = df["positionOrder"].astype(int)

# Criar a variável alvo: 1 se terminou no pódio , 0 caso contrário
df["Podium"] = (df["positionOrder"] <= 3).astype(int)

# Criar variável "ganho de posições": grid - posição final
df["gained_positions"] = df["grid"] - df["positionOrder"]

# Ver as primeiras linhas do dataset final
df.head()

## 2. Seleção de Variáveis para Clustering

Como queremos agrupar perfis de desempenho em corrida, vamos focar-nos em variáveis numéricas que descrevem o que aconteceu nessa corrida para um dado piloto:

- `grid` – posição de partida na grelha;  
- `positionOrder` – posição final oficial;  
- `gained_positions` – diferença entre posição de partida e chegada (positivo = corrida de recuperação, negativo = perda de lugares);  
- `points` – pontos obtidos na corrida;  
- `laps` – nº de voltas completadas;  
- `milliseconds` – tempo total de corrida;

#### Construção do dataset de clustering

In [ ]:
# Seleção das colunas necessárias para construir o perfil dos pilotos
cols_clustering = [
    "resultId",
    "raceId",
    "driverId",
    "constructorId",
    "year",
    "round",
    "circuitId",
    "country",
    "grid",
    "positionOrder",
    "gained_positions",
    "points",
    "laps",
    "milliseconds",
    "Podium"
]

df_clust = df[cols_clustering].copy()

df_clust.head()

#### Tratamento de valores em falta

In [ ]:
# Verificar valores em falta nas colunas numéricas de interesse
numeric_cols = ["grid", "positionOrder", "gained_positions", "points", "laps", "milliseconds"]
df_clust[numeric_cols].isna().sum()

In [ ]:
# Remover linhas com valores em falta nas colunas numéricas
df_clust_clean = df_clust.dropna(subset=numeric_cols).copy()

print("Linhas originais:", df_clust.shape[0])
print("Linhas após remoção de NaN nas features:", df_clust_clean.shape[0])

df_clust_clean.head()

#### Definir matriz X e normalizar

In [ ]:
# Features numéricas para clustering
features = ["grid", "positionOrder", "gained_positions", "points", "laps", "milliseconds"]

X = df_clust_clean[features].copy()

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled[:5]

## 3. Escolha do Número de Clusters (K) – K-Means

Vamos aplicar o algoritmo **K-Means** para diferentes valores de K.

Para cada K vamos calcular:

- **Inertia** – Soma das distâncias ao centroide;  
- **Silhouette Score** – Mede o quão bem separados e coesos estão os clusters.

Com base nestes gráficos, vamos escolher um valor de K “razoável” e usá-lo para treinar o modelo final de K-Means.


#### Testar vários K, calcular inertia e silhouette

In [ ]:
# Testar diferentes valores de K para KMeans
range_k = range(2, 9)

inertias = []
silhouette_scores = []

for k in range_k:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_k = kmeans.fit_predict(X_scaled)
    
    inertia = kmeans.inertia_
    inertias.append(inertia)
    
    sil_score = silhouette_score(X_scaled, labels_k)
    silhouette_scores.append(sil_score)
    
    print(f"K = {k}: inertia = {inertia:.2f}, silhouette = {sil_score:.4f}")

#### Gráficos Elbow e Silhouette vs K

In [ ]:
# Plot dos resultados
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Elbow (inertia)
ax[0].plot(list(range_k), inertias, marker="o")
ax[0].set_xlabel("K")
ax[0].set_ylabel("Inertia (SSE)")
ax[0].set_title("Método Elbow - KMeans")

# Silhouette
ax[1].plot(list(range_k), silhouette_scores, marker="o")
ax[1].set_xlabel("K")
ax[1].set_ylabel("Silhouette Score")
ax[1].set_title("Silhouette vs K - KMeans")

plt.tight_layout()
plt.show()

#### Escolher K com base na Silhouette

In [ ]:
# Melhor K segundo a Silhouette
best_k_index = np.argmax(silhouette_scores)
best_k = list(range_k)[best_k_index]

print(f"Melhor K segundo a Silhouette: {best_k}")

#### K-Means final com K escolhido

In [ ]:
# Treinar KMeans com o melhor K encontrado
k = best_k  
kmeans_final = KMeans(n_clusters=k, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_scaled)

sil_final = silhouette_score(X_scaled, cluster_labels)
print(f"Silhouette Score final (KMeans, k={k}): {sil_final:.4f}")

#### Adicionar clusters ao dataset

In [ ]:
# Adicionar a coluna de cluster ao dataset
df_clusters = df_clust_clean.copy()
df_clusters["cluster"] = cluster_labels

df_clusters.head()

## 4. Análise dos Clusters

Depois de atribuir um cluster a cada resultado (piloto-corrida), vamos analisar:

- Quantos registos existem em cada cluster;
- As estatísticas básicas (média, desvio padrão, mínimo, máximo) das variáveis numéricas por cluster;
- A distribuição de algumas variáveis categóricas, como:
  - pódios (`Podium`);
  - equipas (`constructorId`);
  - país do circuito (`country`).

Isto vai permitir interpretar cada cluster como um certo tipo de corrida.

#### Tamanho dos clusters

In [ ]:
# Contagem de elementos por cluster
df_clusters["cluster"].value_counts().sort_index()

#### Estatísticas numéricas por cluster

In [ ]:
# Estatísticas descritivas por cluster
cluster_stats = (
    df_clusters
    .groupby("cluster")[features]
    .agg(["mean", "std", "min", "max"])
    .round(3)
)

cluster_stats

#### Distribuição de pódios por cluster

In [ ]:
# Tabela de contingência: cluster vs Podium
podium_crosstab = pd.crosstab(df_clusters["cluster"], df_clusters["Podium"],
                              rownames=["Cluster"], colnames=["Podium"])

podium_crosstab

#### Juntar nome da equipa para análise

In [ ]:
# Juntar nome da equipa para análise
df_clusters = df_clusters.merge(
    constructors[["constructorId", "name", "nationality"]],
    on="constructorId",
    how="left",
    suffixes=("", "_constructor")
)

df_clusters.rename(columns={"name": "constructor_name",
                            "nationality": "constructor_nationality"}, inplace=True)

df_clusters[["constructorId", "constructor_name", "constructor_nationality"]].head()

#### Equipas mais frequentes por cluster

In [ ]:
# Top 5 equipas por cluster 
for c in sorted(df_clusters["cluster"].unique()):
    print(f"\n=== Cluster {c} ===")
    top_teams = (
        df_clusters[df_clusters["cluster"] == c]["constructor_name"]
        .value_counts()
        .head(3)
    )
    print(top_teams)


#### PCA para visualização 2D dos clusters

In [ ]:
# Aplicar PCA para visualização dos clusters
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

print("Variância explicada pelas 2 componentes principais:",
      pca.explained_variance_ratio_.sum())

In [ ]:
# Visualizar clusters
plt.figure(figsize=(10, 6))

for cluster_id in sorted(df_clusters["cluster"].unique()):
    mask = (df_clusters["cluster"] == cluster_id)
    plt.scatter(
        X_pca[mask, 0],
        X_pca[mask, 1],
        alpha=0.6,
        label=f"Cluster {cluster_id}"
    )

plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title(f"Clusters de perfis de corrida (KMeans, k={k}) – PCA 2D")
plt.legend()
plt.tight_layout()
plt.show()

## 5. Conclusão

### 5.1. Resultados do clustering (K-Means)

Depois de preparado o dataset, foram selecionadas 6003 corridas com informação completa para aplicar o algoritmo K-Means.

Foi testado vários valores de `k` e, de acordo com o **silhouette score**, o melhor resultado foi obtido com **k = 3**. Com este valor, os clusters ficaram distribuídos da seguinte forma:

- **Cluster 0** → 3 676 corridas (61%)  
- **Cluster 1** → 124 corridas (2%)  
- **Cluster 2** → 2 203 corridas (37%)

A projeção em 2D com PCA retém cerca de 67% da variância total, permitindo visualizar de forma razoável a separação entre os três grupos, com o cluster 0 a destacar-se claramente dos restantes.

---

### 5.2. Caracterização resumida dos clusters

De forma resumida, os clusters podem ser interpretados da seguinte forma:

- **Cluster 0 – Corridas “dominantes” / equipas de topo**  
  - `grid` e `positionOrder` médios baixos, partem à frente e terminam à frente;  
  - elevada percentagem de pódios (`Podium = 1` em muitos casos);  
  - dominado por construtores como **Ferrari, McLaren, Williams, Red Bull e Mercedes**;  
  - representa o cenário de boas performances de equipas fortes.

- **Cluster 1 – Corridas “históricas/longas” com grandes recuperações**  
  - grelhas grandes (`grid` médio elevado);  
  - grandes ganhos de posições (`gained_positions` elevado);  
  - presença de construtores clássicos como **Kurtis Kraft, Epperly, Kuzma, Watson, Phillips**, associados a corridas mais antigas e atípicas no contexto da F1;  
  - funciona praticamente como um cluster de corridas **históricas/outliers**.

- **Cluster 2 – Corridas de “recuperação / meio da tabela”**  
  - partida mais a meio/atrás na grelha (`grid` em torno de 12);  
  - recuperações moderadas (`gained_positions` em média positivo, mas menor que no cluster 1);  
  - maioria das corridas na zona de pontos, com pódios ocasionais;  
  - forte presença de equipas como **McLaren, Williams, Ferrari, Sauber e Force India**, mas com desempenhos menos dominantes do que no cluster 0.

---

### 5.3. Conclusão final

Resumindo, o clustering permitiu:

- Identificar três perfis distintos de desempenho em corrida:  
  - corridas dominantes de equipas de topo (cluster 0);  
  - corridas históricas/atípicas e muito longas, com grandes recuperações (cluster 1);  
  - corridas de recuperação e meio da tabela, com resultados mais modestos (cluster 2).
- Mostrar que é possível extrair estrutura útil dos dados de F1 apenas com variáveis numéricas de desempenho

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1) Ano vs Pontos
axes[0, 0].scatter(
    df_clusters["year"],
    df_clusters["points"],
    c=df_clusters["cluster"],
    alpha=0.6
)
axes[0, 0].set_xlabel("Ano")
axes[0, 0].set_ylabel("Pontos")
axes[0, 0].set_title("Pontos vs Ano")
axes[0, 0].grid(True, alpha=0.3)

# 2) Grid vs Posição final
axes[0, 1].scatter(
    df_clusters["grid"],
    df_clusters["positionOrder"],
    c=df_clusters["cluster"],
    alpha=0.6
)
axes[0, 1].set_xlabel("Posição na grelha")
axes[0, 1].set_ylabel("Posição final")
axes[0, 1].set_title("Grid vs Posição Final")
axes[0, 1].grid(True, alpha=0.3)

# 3) Lugares ganhos vs Pontos
axes[1, 0].scatter(
    df_clusters["gained_positions"],
    df_clusters["points"],
    c=df_clusters["cluster"],
    alpha=0.6
)
axes[1, 0].set_xlabel("Lugares ganhos/perdidos")
axes[1, 0].set_ylabel("Pontos")
axes[1, 0].set_title("Ganho de posições vs Pontos")
axes[1, 0].grid(True, alpha=0.3)

# 4) Distribuição dos clusters por década
df_clusters["decade_cat"] = ((df_clusters["year"] // 10) * 10).astype(str) + "s"

decade_cluster = (
    pd.crosstab(df_clusters["decade_cat"], df_clusters["cluster"], normalize="index") * 100
)

decade_cluster.plot(
    kind="bar",
    stacked=True,
    ax=axes[1, 1]
)

axes[1, 1].set_xlabel("Década")
axes[1, 1].set_ylabel("Percentagem")
axes[1, 1].set_title("Distribuição dos clusters por década")
axes[1, 1].legend(title="Cluster")
axes[1, 1].set_xticklabels(axes[1, 1].get_xticklabels(), rotation=45)

plt.tight_layout()
plt.show()
